# Heat System with Storage

District heating with a thermal storage tank and time-varying gas prices.

This notebook introduces:

- **Storage**: Thermal buffer with charge/discharge flows, efficiency, and self-discharge
- **Time-varying prices**: Cost coefficients that change per timestep
- **Load shifting**: The optimizer charges storage when gas is cheap and discharges when expensive

## Setup

In [ ]:
import math
from datetime import datetime, timedelta

import plotly.express as px
import plotly.io as pio
import polars as pl

from fluxopt import Bus, Converter, Effect, Flow, Port, Storage, solve

pio.renderers.default = 'notebook_connected'

## Generate Time Series Data

We model 48 hours with hourly resolution. Heat demand follows a day/night
pattern, and gas prices have off-peak/peak tariffs.

In [ ]:
n_hours = 48
timesteps = pl.datetime_range(
    datetime(2024, 1, 15), datetime(2024, 1, 15) + timedelta(hours=n_hours - 1), interval='1h', eager=True
)

# Heat demand: sinusoidal pattern - peaks during business hours, low at night
heat_demand = [50 + 40 * max(0.0, math.sin(math.pi * ((h % 24) - 6) / 12)) for h in range(n_hours)]

# Gas price: off-peak (22:00-06:00) at 0.04 EUR/kWh, peak at 0.08 EUR/kWh
gas_price = [0.04 if (h % 24) < 6 or (h % 24) >= 22 else 0.08 for h in range(n_hours)]

Visualize the demand and price profiles:

In [ ]:
px.line(x=timesteps, y=heat_demand, labels={'x': 'Time', 'y': 'kW'}, title='Heat Demand')

In [ ]:
px.line(x=timesteps, y=gas_price, labels={'x': 'Time', 'y': 'EUR/kWh'}, title='Gas Price')

## Build and Solve the Energy System

The system:

```
Gas Grid ──► [gas] ──► Boiler ──► [heat] ◄──► Storage
                         η=92%       │
                                     ▼
                                   Office
```

In [ ]:
# Gas supply with time-varying price
gas_source = Flow(bus='gas', size=500, effects_per_flow_hour={'cost': gas_price})

# Boiler: 150 kW thermal capacity, 92% efficiency
fuel = Flow(bus='gas', size=300)
heat_out = Flow(bus='heat', size=150)

# Demand: normalize to relative profile
max_demand = max(heat_demand)
demand_profile = [d / max_demand for d in heat_demand]
demand_flow = Flow(bus='heat', size=max_demand, fixed_relative_profile=demand_profile)

# Thermal storage: 500 kWh, 98% charge/discharge efficiency, 0.5%/h self-discharge
charge_flow = Flow(bus='heat', size=100)
discharge_flow = Flow(bus='heat', size=100)
storage = Storage(
    'thermal_storage',
    charging=charge_flow,
    discharging=discharge_flow,
    capacity=500,
    eta_charge=0.98,
    eta_discharge=0.98,
    relative_loss_per_hour=0.005,
    initial_charge_state=0.5,  # start half-full (fraction of capacity)
)

result = solve(
    timesteps=timesteps,
    buses=[Bus('gas'), Bus('heat')],
    effects=[Effect('cost', unit='€', is_objective=True)],
    ports=[
        Port('gas_grid', imports=[gas_source]),
        Port('office', exports=[demand_flow]),
    ],
    converters=[
        Converter.boiler('boiler', thermal_efficiency=0.92, fuel_flow=fuel, thermal_flow=heat_out),
    ],
    storages=[storage],
)

## Analyze Results

### Cost Summary

In [ ]:
total_heat = sum(heat_demand)

pl.DataFrame(
    {
        'Metric': ['Total heat delivered', 'Total cost', 'Avg cost of heat'],
        'Value': [total_heat, result.objective, result.objective / total_heat * 100],
        'Unit': ['kWh', 'EUR', 'ct/kWh'],
    }
)

### Flow Rates

All flow rates across the horizon:

In [ ]:
px.line(
    result.flow_rates,
    x='time',
    y='solution',
    color='flow',
    title='Flow Rates',
    labels={'solution': 'kW'},
    line_shape='hv',
)

### Storage Operation

Charge and discharge rates show the load-shifting behavior:

In [ ]:
storage_flows = result.flow_rates.filter(pl.col('flow').str.starts_with('thermal_storage'))
px.line(
    storage_flows,
    x='time',
    y='solution',
    color='flow',
    title='Storage Charge / Discharge',
    labels={'solution': 'kW'},
    line_shape='hv',
)

### Charge State

The stored energy over time:

In [ ]:
cs = result.charge_state('thermal_storage')
px.area(cs, x='time', y='solution', title='Storage Charge State', labels={'solution': 'kWh'}, line_shape='hv')

### Heat Balance

How boiler output and storage work together to meet demand:

In [ ]:
heat_flows = result.flow_rates.filter(pl.col('flow').str.contains('heat|charge|discharge'))
px.line(
    heat_flows,
    x='time',
    y='solution',
    color='flow',
    title='Heat Bus Balance',
    labels={'solution': 'kW'},
    line_shape='hv',
)

### Effect Contributions

Breakdown of which flows drive cost:

In [ ]:
px.bar(
    result.contributions,
    x='time',
    y='solution',
    color='contributor',
    title='Cost Contributions by Flow',
    labels={'solution': 'EUR'},
)

## Key Insights

The optimization reveals how storage enables **load shifting**:

1. **Charge during off-peak**: When gas is cheap (night), the boiler runs at
   higher output to charge the storage
2. **Discharge during peak**: During expensive periods, storage supplements the
   boiler, reducing gas purchases
3. **Efficiency trade-off**: Storage has round-trip losses (0.98 × 0.98 = 96%)
   plus self-discharge, so shifting only happens when the price spread exceeds
   the loss cost

## Summary

You learned how to:

- Add **Storage** with capacity, efficiency, and self-discharge
- Use **time-varying prices** via `effects_per_flow_hour`
- Inspect **charge states** and **storage operation**
- Join result DataFrames for combined analysis